In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub


import torch
device = torch.device("cuda")

In [3]:
import numpy as np 
import pandas as pd
data_train1 = pd.read_csv("../input/nlp-getting-started/train.csv")
data_test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [4]:
data_train2 = pd.read_csv("../input/disaster-tweets/tweets.csv")

In [5]:
#print(data_train.columns)
print(data_test.columns)
#print(data_train.shape)
print(data_test.shape)

In [6]:
#print(data_train.columns)
print(data_train1.columns)
frames = [data_train1, data_train2]
data_train = pd.concat(frames)

In [7]:
data_train.shape

In [8]:
data_train['target'].value_counts()

In [9]:
import tensorflow_hub as hub
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TFBertForSequenceClassification,TFRobertaForSequenceClassification
module_url = TFRobertaForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli', output_hidden_states=True)
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [11]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    
    transformer = bert_layer([input_word_ids, input_mask, segment_ids])
    hidden_states = transformer[1]
    hidden_states_size = 4 # count of the last states 
    hiddes_states_ind = list(range(-hidden_states_size, 0, 1))
    selected_hidden_states = tf.keras.layers.concatenate(tuple([hidden_states[i] for i in hiddes_states_ind]))
    print(selected_hidden_states.shape)
    
    out = tf.keras.layers.Dense(1, activation='sigmoid')(selected_hidden_states)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [12]:
from tensorflow import keras
max_len = 96
train_input = bert_encode(data_train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(data_test.text.values, tokenizer, max_len=max_len)
#train_labels = tf.keras.utils.to_categorical(data_train.target.values, num_classes=2, dtype='float32')
train_labels = data_train.target.values

In [13]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

In [14]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, restore_best_weights=True)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=10,
    callbacks=[earlystopping],
    batch_size=16)

In [15]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning_intro.ex6 import *

In [16]:
history_df = pd.DataFrame(train_history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
#history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

In [17]:
test_pred = model.predict(test_input)

In [18]:
test_pred

In [19]:

preds = [np.mean(i).round().astype(int) for i in test_pred]

In [20]:
len(preds)

In [21]:
preds

In [22]:
submission  = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

In [23]:
submission.columns

In [24]:
submission['target'] = preds

In [25]:
submission['target'].value_counts()

In [26]:
submission.to_csv("submission.csv", index = False)